# 4. Prueba de consultas

En esta seccion probamos las consultas

In [2]:
import os
import pandas as pd

In [3]:
data_mvp_funciones = pd.read_csv(os.path.join('D:\Bootcamp\Henry\CURSO\Proyecto Individual 1\Movies\Pipeline','data_movies_subset_limpia.csv'),index_col=0).convert_dtypes()
data_mvp_funciones_final = pd.read_csv(os.path.join('D:\Bootcamp\Henry\CURSO\Proyecto Individual 1\Movies\Output','data_mvp_final_ml.csv'),index_col=0).convert_dtypes()
data_mvp_director = pd.read_csv(os.path.join('D:\Bootcamp\Henry\CURSO\Proyecto Individual 1\Movies\Output','data_mvp_final_funciones_exitodir.csv'),index_col=0).convert_dtypes()
data_mvp_recomendacion = pd.read_csv(os.path.join('D:\Bootcamp\Henry\CURSO\Proyecto Individual 1\Movies\Output','data_mvp_ml_recomend_indexed.csv'),index_col=[0,1]).convert_dtypes()
data_mvp_actor = pd.read_csv(os.path.join('D:\Bootcamp\Henry\CURSO\Proyecto Individual 1\Movies\Pipeline','credits_normalizada.csv'),index_col=0).convert_dtypes()

### 1. def cantidad_filmaciones_mes( Mes ): 
Se ingresa un mes en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en el mes consultado en la totalidad del dataset.

In [4]:
def cantidad_filmaciones_mes(mes: str):
    # Crear un diccionario para convertir los meses en español a números
    meses = {
        'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4,
        'mayo': 5, 'junio': 6, 'julio': 7, 'agosto': 8,
        'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
    }
    
    # Convertir el mes a minúsculas para asegurar la coincidencia
    mes = mes.lower()
    
    if mes not in meses:
        raise ValueError("Mes inválido. Por favor ingrese un mes válido en español.")
    
    # Obtener el número correspondiente al mes
    mes_numero = meses[mes]
    
    # Convertir la columna de fechas a datetime si no lo está ya
    data_mvp_funciones ['release_date'] = pd.to_datetime(data_mvp_funciones['release_date'], errors='coerce')
    
    # Filtrar el DataFrame por el mes
    peliculas_mes = data_mvp_funciones[data_mvp_funciones['release_date'].dt.month == mes_numero]
    
    # Devolver la cantidad de filmaciones en ese mes
    return f"{len(peliculas_mes)} cantidad de peliculas fueron estrenadas en el mes de {mes}"

In [5]:
print(cantidad_filmaciones_mes('diciembre'))

3781 cantidad de peliculas fueron estrenadas en el mes de diciembre


### 2. def cantidad_filmaciones_dia( Dia ): 
Se ingresa un día en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en día consultado en la totalidad del dataset.

In [6]:
def cantidad_filmaciones_dia(dia: str):
    # Crear un diccionario para convertir los días de la semana en español a números
    dias_semana = {
        'lunes': 0, 'martes': 1, 'miércoles': 2, 'jueves': 3,
        'viernes': 4, 'sábado': 5, 'domingo': 6
    }
    
    # Convertir el día a minúsculas para asegurar la coincidencia
    dia = dia.lower()
    
    if dia not in dias_semana:
        raise ValueError("Día inválido. Por favor ingrese un día válido en español.")
    
    # Obtener el número correspondiente al día
    dia_numero = dias_semana[dia]
    
    # Convertir la columna de fechas a datetime si no lo está ya
    data_mvp_funciones['release_date'] = pd.to_datetime(data_mvp_funciones['release_date'], errors='coerce')
    
    # Filtrar el DataFrame por el día de la semana
    peliculas_dia = data_mvp_funciones[data_mvp_funciones['release_date'].dt.dayofweek == dia_numero]
    
    # Devolver la cantidad de filmaciones en ese día
    return f'{len(peliculas_dia)} cantidad de peliculas fueron estrenadas en los dias {dia}'

In [7]:
print(cantidad_filmaciones_dia('lunes'))

3500 cantidad de peliculas fueron estrenadas en los dias lunes


### 3. def score_titulo( titulo_de_la_filmación ): 
Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score.

In [8]:
def score_titulo(titulo_de_la_filmacion):
    # Convertir el título ingresado a minúsculas
    titulo_de_la_filmacion = titulo_de_la_filmacion.lower()
    
    # Filtrar el dataset buscando coincidencias en el título
    resultados = data_mvp_funciones[data_mvp_funciones['title'].str.lower().str.contains(titulo_de_la_filmacion, na=False)]
    
    # Crear una lista de diccionarios con el formato {'título': title, 'año': release_year, 'popularidad': popularity}
    lista_resultados = [
        {'Título': row['title'], 'Año de estreno': row['release_year'], 'Popularidad': row['popularity']} 
        for _, row in resultados.iterrows()
    ]
    
    return lista_resultados

# Ejemplo de uso
# Supongamos que tienes un DataFrame llamado df con las columnas movie_id, title, release_year, y popularity
# lista_peliculas = score_titulo('inception')
# print(lista_peliculas)


In [9]:
score_titulo('toy story')

[{'Título': 'Toy Story', 'Año de estreno': 1995, 'Popularidad': 21.946943},
 {'Título': 'Toy Story 2', 'Año de estreno': 1999, 'Popularidad': 17.547693},
 {'Título': 'Toy Story 3', 'Año de estreno': 2010, 'Popularidad': 16.96647},
 {'Título': 'Toy Story of Terror!',
  'Año de estreno': 2013,
  'Popularidad': 0.512025},
 {'Título': 'Toy Story That Time Forgot',
  'Año de estreno': 2014,
  'Popularidad': 8.609555}]

### 4. def votos_titulo( titulo_de_la_filmación ): 
Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. La misma variable deberá de contar con al menos 2000 valoraciones, caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.

In [10]:
def votos_titulo(titulo_de_la_filmacion):
    # Filtrar por coincidencias en el título, ignorando mayúsculas/minúsculas
    coincidencias = data_mvp_funciones[data_mvp_funciones['title'].str.contains(titulo_de_la_filmacion, case=False, na=False)]
    
    # Lista para almacenar los resultados
    resultados = []
    
    # Recorrer las coincidencias y filtrar por aquellas que tienen al menos 2000 votos
    for _, row in coincidencias.iterrows():
        if row['vote_count'] >= 2000:
            resultado = {
                'Título': row['title'],
                'Año de estreno': row['release_year'],
                'Cantidad de votos': row['vote_count'],
                'Puntaje promedio': row['vote_average']
            }
            resultados.append(resultado)
    
    # Verificar si se encontraron películas que cumplan con la condición
    if resultados:
        return resultados
    else:
        return "No se encontraron películas con al menos 2000 valoraciones."



In [11]:

votos_titulo('Toy Story')

[{'Título': 'Toy Story',
  'Año de estreno': 1995,
  'Cantidad de votos': 5415,
  'Puntaje promedio': 7.7},
 {'Título': 'Toy Story 2',
  'Año de estreno': 1999,
  'Cantidad de votos': 3914,
  'Puntaje promedio': 7.3},
 {'Título': 'Toy Story 3',
  'Año de estreno': 2010,
  'Cantidad de votos': 4710,
  'Puntaje promedio': 7.6}]

### 5. def get_actor( nombre_actor ): 
Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, la cantidad de películas que en las que ha participado y el promedio de retorno. La definición no deberá considerar directores.

In [12]:
def get_actor(nombre_actor:str):
    # Filtrar el DataFrame de cast para obtener las filas donde aparece el actor
    peliculas_con_actor = data_mvp_actor[data_mvp_actor['prtgnst_name'].str.contains(nombre_actor, case=False, na=False)]
    
    if peliculas_con_actor.empty:
        return f"No se encontraron películas para el actor: {nombre_actor}"
    
    # Eliminar duplicados: quedarse con una sola fila por combinación de movie_id y actor
    peliculas_con_actor = peliculas_con_actor.drop_duplicates(subset=['movie_id', 'prtgnst_name'])
    
    # Unir el DataFrame de películas con el DataFrame de cast usando movie_id
    df_actor_movies = peliculas_con_actor.merge(data_mvp_funciones, on='movie_id', how='inner')
    
    # Calcular la cantidad de películas en las que ha participado el actor
    cantidad_peliculas = df_actor_movies.shape[0]
    
    # Calcular el retorno total y el promedio de retorno para las películas en las que ha participado
    retorno_total = df_actor_movies['return'].sum()
    promedio_retorno = df_actor_movies['return'].mean()
    
    return f'El actor {nombre_actor} ha participado en {cantidad_peliculas} cantidad de filmaciones, el mismo a conseguido un retorno de {round(retorno_total,2)} con un promedio {round(promedio_retorno,2)} por filmación.'

In [13]:
get_actor('tom hanks')

'El actor tom hanks ha participado en 48 cantidad de filmaciones, el mismo a conseguido un retorno de 166.16 con un promedio 4.26 por filmación.'

### 6. def get_director( nombre_director ): 
Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.

In [14]:
def get_director(nombre_director: str):
    director_subset = data_mvp_director.loc[
        data_mvp_director['director'].str.contains(nombre_director,case=False, na=False)]
    director_return = director_subset['director_return'].drop_duplicates()
    director_return_res = round(float(director_return.iloc[0]),2)
    columns_peliculas=['movie_id','title','release_date','budget','revenue','return']
    director_peliculas = director_subset[columns_peliculas].drop_duplicates().fillna(0) ### OJO se colocó fillna(0) para corregir el error de NAType
    peliculas = []
    for _, pelicula in director_peliculas.iterrows():
        pelicula_info = {
            'nombre': pelicula['title'],
            'fecha_lanzamiento': pelicula['release_date'],
            'costo (USD)': ('{:,}'.format(round(pelicula['budget']))),
            'ganancia (USD)': ('{:,}'.format(round(pelicula['revenue']))),
            'retorno (ratio)': round(pelicula['return'],2)
        }
        peliculas.append(pelicula_info)
    return {
        'director': nombre_director,
        'retorno total': director_return_res,
        'peliculas': peliculas }


In [18]:
# test
get_director('james cameron')

{'director': 'james cameron',
 'retorno total': 7.76,
 'peliculas': [{'nombre': 'True Lies',
   'fecha_lanzamiento': '1994-07-14',
   'costo (USD)': '115,000,000',
   'ganancia (USD)': '378,882,411',
   'retorno (ratio)': 3.29},
  {'nombre': 'Terminator 2: Judgment Day',
   'fecha_lanzamiento': '1991-07-01',
   'costo (USD)': '100,000,000',
   'ganancia (USD)': '520,000,000',
   'retorno (ratio)': 5.2},
  {'nombre': 'The Abyss',
   'fecha_lanzamiento': '1989-08-09',
   'costo (USD)': '70,000,000',
   'ganancia (USD)': '90,000,098',
   'retorno (ratio)': 1.29},
  {'nombre': 'Aliens',
   'fecha_lanzamiento': '1986-07-18',
   'costo (USD)': '18,500,000',
   'ganancia (USD)': '183,316,455',
   'retorno (ratio)': 9.91},
  {'nombre': 'The Terminator',
   'fecha_lanzamiento': '1984-10-26',
   'costo (USD)': '6,400,000',
   'ganancia (USD)': '78,371,200',
   'retorno (ratio)': 12.25},
  {'nombre': 'Titanic',
   'fecha_lanzamiento': '1997-11-18',
   'costo (USD)': '200,000,000',
   'ganancia (U

### 7. def recomendacion(titulo): 
Se ingresa el nombre de una película y te recomienda las similares en una lista de 5 valores.

In [16]:
#@app.get("/recomendacion_pelicula/{titulo_pelicula}")
# definimos
def recomendacion_pelicula(titulo_pelicula: str):
    recomendacion_subset = data_mvp_recomendacion.loc[
        data_mvp_recomendacion['title'].isin([titulo_pelicula])]
    pelicula_vecinos = recomendacion_subset[['vecino_title']].reset_index().drop_duplicates()
    pelicula_recomend = list(pelicula_vecinos['vecino_title'])
    return {
        'pelicula': titulo_pelicula,
        'recomendaciones': pelicula_recomend[:5]}
# test
recomendacion_pelicula('Toy Story')

{'pelicula': 'Toy Story',
 'recomendaciones': ['Class of 1984',
  'Totally Fucked Up',
  'Dollman vs. Demonic Toys',
  'Halloweentown',
  "City Slickers II: The Legend of Curly's Gold"]}